In [101]:
import torch
import torch.nn as nn # use to make the weight and bias tensors
import torch.nn.functional as F # give activation functions
from torch.optim import SGD # short for Stochastic Gradient Descent, to fit the neural network to the data

# draw nice looking graphs
import matplotlib.pyplot as plt
import seaborn as sns

import random
import math


In [105]:
import pandas as pd
import numpy as np
bot_detectoin = pd.read_csv('/Users/noah/Desktop/Jupyter_Notebook/archive/bot_detection_data.csv')
print(bot_detectoin.head())
print(bot_detectoin.info())

   User ID        Username                                              Tweet  \
0   132131           flong  Station activity person against natural majori...   
1   289683  hinesstephanie  Authority research natural life material staff...   
2   779715      roberttran  Manage whose quickly especially foot none to g...   
3   696168          pmason  Just cover eight opportunity strong policy which.   
4   704441          noah87                      Animal sign six data good or.   

   Retweet Count  Mention Count  Follower Count  Verified  Bot Label  \
0             85              1            2353     False          1   
1             55              5            9617      True          0   
2              6              2            4363      True          0   
3             54              5            2242      True          1   
4             26              3            8438     False          1   

       Location           Created At            Hashtags  
0     Adkinston  2020

In [108]:

# Convert 'Created At' column to datetime format
bot_detectoin['Created At'] = pd.to_datetime(bot_detectoin['Created At'])

num_hours = 24
num_months = 12

# Define a function to calculate the sine of the hour component
def calculate_sine_hour(created_at):
    hour = created_at.hour
    sine_hour = np.sin(2 * np.pi * hour / num_hours)
    return sine_hour

def calculate_cosine_hour(created_at):
    hour = created_at.hour
    cosine_hour = np.cos(2 * np.pi * hour / num_hours)
    return cosine_hour

def calculate_sine_month(month):
    return np.sin(2 * np.pi * month / num_months)

def calculate_cosine_month(month):
    return np.cos(2 * np.pi * month / num_months)


# Create a new column 'Sine Hour' using apply() and a lambda function
bot_detectoin['Sine Hour'] = bot_detectoin['Created At'].apply(lambda x: calculate_sine_hour(x))
bot_detectoin['Cosine Hour'] = bot_detectoin['Created At'].apply(lambda x: calculate_cosine_hour(x))
bot_detectoin['Sine Month'] = bot_detectoin['Created At'].dt.month.apply(calculate_sine_month)
bot_detectoin['Cosine Month'] = bot_detectoin['Created At'].dt.month.apply(calculate_cosine_month)

bot_detectoin['Verified_Num'] = bot_detectoin['Verified'].astype(int)


# Print the DataFrame with the new column
print(bot_detectoin)

       User ID         Username  \
0       132131            flong   
1       289683   hinesstephanie   
2       779715       roberttran   
3       696168           pmason   
4       704441           noah87   
...        ...              ...   
49995   491196            uberg   
49996   739297     jessicamunoz   
49997   674475   lynncunningham   
49998   167081  richardthompson   
49999   311204         daniel29   

                                                   Tweet  Retweet Count  \
0      Station activity person against natural majori...             85   
1      Authority research natural life material staff...             55   
2      Manage whose quickly especially foot none to g...              6   
3      Just cover eight opportunity strong policy which.             54   
4                          Animal sign six data good or.             26   
...                                                  ...            ...   
49995  Want but put card direction know miss former h.

In [123]:
# Assuming 'bot_detection' is your dataset
# Extract the columns you want to standardize
retweet_count = bot_detectoin['Retweet Count']
mention_count = bot_detectoin['Mention Count']
follower_count = bot_detectoin['Follower Count']

# Calculate mean and standard deviation for each column
retweet_mean = retweet_count.mean()
retweet_std = retweet_count.std()

mention_mean = mention_count.mean()
mention_std = mention_count.std()

follower_mean = follower_count.mean()
follower_std = follower_count.std()

# Apply Z-score normalization to each column
bot_detectoin['Retweet Count Normalized'] = (retweet_count - retweet_mean) / retweet_std
bot_detectoin['Mention Count Normalized'] = (mention_count - mention_mean) / mention_std
bot_detectoin['Follower Count Normalized'] = (follower_count - follower_mean) / follower_std


In [130]:
from sklearn.model_selection import train_test_split

selected_columns = ['Retweet Count Normalized', 'Follower Count Normalized', 'Verified_Num', 'Sine Hour', 'Cosine Hour', 'Sine Month', 'Cosine Month', 'Bot Label']
bot_detection_clean = bot_detectoin[selected_columns]

# Split the dataset into features (X) and labels (y)
X = bot_detection_clean.drop(columns=['Bot Label'])  # Features
y = bot_detection_clean['Bot Label']  # Target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)




       Retweet Count Normalized  Follower Count Normalized  Verified_Num  \
39087                 -1.439477                  -1.163217             1   
30893                  1.576168                   1.557068             1   
45278                 -1.645089                  -0.025568             1   
16398                  1.541899                   1.283684             0   
13653                 -1.199596                   1.704702             0   
...                         ...                        ...           ...   
11284                 -1.542283                  -0.811675             0   
44732                  0.822257                  -1.557486             0   
38158                  0.548107                   0.635138             1   
860                   -0.308610                   1.124587             1   
15795                  0.548107                  -0.017578             0   

       Sine Hour   Cosine Hour    Sine Month  Cosine Month  
39087  -0.866025 -5.000000

In [131]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
    
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
    
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
      
        return out
  
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out
  
    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)

    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        if x > 20:  # Prevent overflow for large positive values
            out_data = 1.0
        elif x < -20:  # Prevent overflow for large negative values
            out_data = -1.0
        else:
            t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
            out_data = t

        out = Value(out_data, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - out.data**2) * out.grad
        out._backward = _backward

        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
    
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
            build_topo(self)
    
            self.grad = 1.0
            for node in reversed(topo):
                node._backward()

In [132]:

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [ ]:

# Training data X_train, y_train are assumed to be defined

input_dim = len(X_train.columns)
n = MLP(nin=input_dim, nouts=[4, 4, 1])



# Training loop
for k in range(20):
    total_loss = 0
   
    # Forward pass
    y_true_values = [Value(value) for value in y_train]
    ys = [n(x) for x in X_train.values]
    loss = sum((y_true - y_pred) ** 2 for y_true, y_pred in zip(y_true_values, ys))

    # Backward pass
    loss.backward()
        
        # Update parameters
    for p in n.parameters():
        p.data -= 0.1 * p.grad
        p.grad = 0.0
        
    print(k, loss.data)


0 42221.05680209554
1 42221.05680209554


In [114]:
y_true_values = [Value(value) for value in y_train]
sum((y_true - y_pred) ** 2 for y_true, y_pred in zip(y_true_values, ys))

Value(data=92753.23907527671)

In [ ]:
n.parameters()